### Train Robust MLP with Boston housing dataset

In [ ]:
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import tensorflow.contrib.slim as slim
import matplotlib.pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format = 'retina'
from mlp_reg_class import mlp_reg_class
from sklearn.datasets import load_boston
from util import nzr,get_train_test_datasets,gpusession
print ("Packages loaded")

In [ ]:
RUN_L1_LOSS = 0
RUN_ROBUST_LOSS = 1
RUN_LEAKY_ROBUST_LOSS = 1

### L1 Loss

In [ ]:
if RUN_L1_LOSS:
    rseed = 1
    dataset,dataset_name = load_boston(),'boston' # select dataset to use
    outlier_rates = [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]
    rmses = []
    for outlier_rate in outlier_rates: # with different outlier rates
        x_train,x_test,y_train,y_test = get_train_test_datasets(
            dataset,dataset_name,_test_size=0.333,_outlier_rate=outlier_rate,
            _seed=rseed,_SAVE_MAT=False,_VERBOSE=False) # get training data with outliers
        tf.reset_default_graph(); sess = gpusession()
        tf.set_random_seed(rseed); np.random.seed(rseed) # fix seeds
        MLP = mlp_reg_class(_name='rmlp_%s'%(dataset_name),_x_dim=np.shape(x_train)[1],_y_dim=1,
                             _h_dims=[256,128],_actv=tf.nn.relu,_bn=slim.batch_norm,#slim.batch_norm/None
                             _l2_reg_coef=1e-3,_GPU_ID=2,_L1_LOSS=1,_ROBUST_LOSS=0,_LEAKY_ROBUST_LOSS=0,_VERBOSE=0)
        sess.run(tf.global_variables_initializer()) # initialize variables
        MLP.train(_sess=sess,_x_train=x_train,_y_train=y_train,
                  _lr=1e-3,_batch_size=256,_max_epoch=1e4,_kp=1.0,
                  _LR_SCHEDULE=0,_PRINT_EVERY=0,_PLOT_EVERY=0,
                  _SAVE_TXT=0,_SAVE_BEST_NET=0,_SAVE_FINAL=0,_REMOVE_PREVS=0,
                  _x_dim4plot=5,_x_name4plot='Average number of rooms per dwelling') # train
        rmse = MLP.test(_sess=sess,_x_train=x_train,_y_train=y_train,_x_test=x_test,_y_test=y_test,
                        _PLOT_TRAIN=0,_PLOT_TEST=1,_SAVE_FIG=0,
                        _title_str4data='Outlier rate:[%.1f]'%(outlier_rate),
                        _x_dim4plot=5,_x_name4plot='Average number of rooms per dwelling') # test
        rmses.append(rmse) # collect rmse
        print ("outlier rate:[%.1f] rmse:[%.4f]"%(outlier_rate,rmse))
    print ("Done.")
    # Plot overall RMSE vs. outlier rate 
    plt.plot(outlier_rates,rmses,'ko-')
    plt.xlabel('Outlier Rate',fontsize=13)
    plt.ylabel('RMSE',fontsize=13)
    plt.show()

### Robust loss 

In [ ]:
if RUN_ROBUST_LOSS:
    rseed = 1
    dataset,dataset_name = load_boston(),'boston' # select dataset to use
    outlier_rates = [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]
    rmses = []
    for outlier_rate in outlier_rates: # with different outlier rates
        x_train,x_test,y_train,y_test = get_train_test_datasets(
            dataset,dataset_name,_test_size=0.333,_outlier_rate=outlier_rate,
            _seed=rseed,_SAVE_MAT=False,_VERBOSE=False) # get training data with outliers
        tf.reset_default_graph(); sess = gpusession()
        tf.set_random_seed(rseed); np.random.seed(rseed) # fix seeds
        MLP = mlp_reg_class(_name='rmlp_%s'%(dataset_name),_x_dim=np.shape(x_train)[1],_y_dim=1,
                             _h_dims=[256,128],_actv=tf.nn.relu,_bn=slim.batch_norm,#slim.batch_norm/None
                             _l2_reg_coef=1e-8,_GPU_ID=-1,_L1_LOSS=0,_ROBUST_LOSS=1,_LEAKY_ROBUST_LOSS=0,_VERBOSE=0)
        sess.run(tf.global_variables_initializer()) # initialize variables
        MLP.train(_sess=sess,_x_train=x_train,_y_train=y_train,
                  _lr=1e-3,_batch_size=256,_max_epoch=1e4,_kp=1.0,
                  _LR_SCHEDULE=0,_PRINT_EVERY=0,_PLOT_EVERY=0,
                  _SAVE_TXT=0,_SAVE_BEST_NET=0,_SAVE_FINAL=0,_REMOVE_PREVS=0,
                  _x_dim4plot=5,_x_name4plot='Average number of rooms per dwelling') # train
        rmse = MLP.test(_sess=sess,_x_train=x_train,_y_train=y_train,_x_test=x_test,_y_test=y_test,
                        _PLOT_TRAIN=0,_PLOT_TEST=1,_SAVE_FIG=0,
                        _title_str4data='Outlier rate:[%.1f]'%(outlier_rate),
                        _x_dim4plot=5,_x_name4plot='Average number of rooms per dwelling') # test
        rmses.append(rmse) # collect rmse
        print ("outlier rate:[%.1f] rmse:[%.4f]"%(outlier_rate,rmse))
    print ("Done.")
    # Plot overall RMSE vs. outlier rate 
    plt.plot(outlier_rates,rmses,'ko-')
    plt.xlabel('Outlier Rate',fontsize=13)
    plt.ylabel('RMSE',fontsize=13)
    plt.show()

### Leaky Robust loss

In [ ]:
if RUN_LEAKY_ROBUST_LOSS:
    rseed = 1
    dataset,dataset_name = load_boston(),'boston' # select dataset to use
    outlier_rates = [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]
    rmses = []
    for outlier_rate in outlier_rates: # with different outlier rates
        x_train,x_test,y_train,y_test = get_train_test_datasets(
            dataset,dataset_name,_test_size=0.333,_outlier_rate=outlier_rate,
            _seed=rseed,_SAVE_MAT=False,_VERBOSE=False) # get training data with outliers
        tf.reset_default_graph(); sess = gpusession()
        tf.set_random_seed(rseed); np.random.seed(rseed) # fix seeds
        MLP = mlp_reg_class(_name='rmlp_%s'%(dataset_name),_x_dim=np.shape(x_train)[1],_y_dim=1,
                             _h_dims=[256,128],_actv=tf.nn.relu,_bn=slim.batch_norm,#slim.batch_norm/None
                             _l2_reg_coef=1e-8,_GPU_ID=1,_L1_LOSS=0,_ROBUST_LOSS=0,_LEAKY_ROBUST_LOSS=1,_VERBOSE=0)
        sess.run(tf.global_variables_initializer()) # initialize variables
        MLP.train(_sess=sess,_x_train=x_train,_y_train=y_train,
                  _lr=1e-3,_batch_size=512,_max_epoch=1e4,_kp=1.0,
                  _LR_SCHEDULE=0,_PRINT_EVERY=0,_PLOT_EVERY=0,
                  _SAVE_TXT=0,_SAVE_BEST_NET=0,_SAVE_FINAL=0,_REMOVE_PREVS=0,
                  _x_dim4plot=5,_x_name4plot='Average number of rooms per dwelling') # train
        rmse = MLP.test(_sess=sess,_x_train=x_train,_y_train=y_train,_x_test=x_test,_y_test=y_test,
                        _PLOT_TRAIN=0,_PLOT_TEST=1,_SAVE_FIG=0,
                        _title_str4data='Outlier rate:[%.1f]'%(outlier_rate),
                        _x_dim4plot=5,_x_name4plot='Average number of rooms per dwelling') # test
        rmses.append(rmse) # collect rmse
        print ("outlier rate:[%.1f] rmse:[%.4f]"%(outlier_rate,rmse))
    print ("Done.")
    # Plot overall RMSE vs. outlier rate 
    plt.plot(outlier_rates,rmses,'ko-')
    plt.xlabel('Outlier Rate',fontsize=13)
    plt.ylabel('RMSE',fontsize=13)
    plt.show()